In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import re
import time
import copy



In [19]:

def scroll_page(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.5)
    cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
    driver.execute_script("window.scrollTo(0, 0);")

def next_day_click(driver):
    a = ActionChains(driver)
    m= driver.find_element(By.CLASS_NAME, "date-filter")
    a.move_to_element(m).perform()

    btn = driver.find_elements(By.CSS_SELECTOR, ".date-item.carousel-cell.ng-star-inserted")
    btn[1].click()
    time.sleep(5)

def get_cinema(driver, i):
    #get cinema list
    scroll_page(driver)
    cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
    try:
        cinema = cinema_list[i]
        return cinema
    except:
        return None 

def cinema_click(cinema):
    cinema_button = cinema.find_element(By.CLASS_NAME, "cinema-title")
    cinema_button.click()

def showtime_click(cinema, show_times, index):
    #get all the h4 elements containing Hindi-2D
    hindi_2d_elements = cinema.find_elements(By.XPATH, '//h4[contains(@class, "type-title") and contains(text(), "HINDI- 2D")]')
    for hindi_2d_element in hindi_2d_elements:
        if len(hindi_2d_element.text)==0:
            continue #there might be empty classes
        ul_element = hindi_2d_element.find_element(By.XPATH, './following-sibling::ul[@class="type-time-slots ng-star-inserted"]')
        
        #get time slots for Hindi 2D in that cinema
        slot_elements = ul_element.find_elements(By.XPATH, './/li[@class="ng-star-inserted"]') 
        for slot in slot_elements:
            #pick the first available slot
            if (len(slot.text.strip()) != 0) and (slot.find_element(By.XPATH, './/span').get_attribute("class").split("-")[1] != "default"):
                show_times[index] = {
                    "Theatre" : cinema.find_element(By.CLASS_NAME, "title").text,
                    "Time" : slot.text.strip(),
                    "type" : slot.find_element(By.XPATH, './/span').get_attribute("class").split("-")[1]
                } 
                break
        break
    slot.click()

def wait_banners(driver):

    try:
        time.sleep(2)
        baggage_btn = driver.find_element(By.CLASS_NAME, "custom-popup-container")
        baggage_btn.find_element(By.CSS_SELECTOR, ".custom-popup-button.ng-star-inserted").click()
    except:
        print("no bag")

    try:
        #open the seating
        time.sleep(2)
        wait = WebDriverWait(driver, 1)
        loaded = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "skip-btn")))

        #press the skip button in case some promo comes
        skip_btn = driver.find_element(By.CLASS_NAME, "skip-btn")
        skip_btn.click()

    except:
        print("no skip")

    #wait for the T&C banner to load and click the close button 
    time.sleep(5)
    close_class = driver.find_element(By.CLASS_NAME, "modal-content-header")
    btn = close_class.find_element(By.CLASS_NAME, "ion-android-close")
    btn.click()

def seat_info(driver, show_times, n):
    #extract seats info
    seat_prices_array = [title.find_element(By.XPATH, ".//span").text for title in driver.find_elements(By.CLASS_NAME, "seats-col") if title.find_element(By.XPATH, ".//span").get_attribute("class")=="area hshshs"]
    seat_prices = [int(float(re.search(r'(\d+\.\d+)', seat).group(1))) for seat in seat_prices_array]

    n_seats = 0
    n_seats_available = 0
    rows = driver.find_elements(By.CSS_SELECTOR, ".seats-row.ng-star-inserted")
    for row in rows:
        n_seats+= len(row.find_elements(By.CSS_SELECTOR, ".seats-col.ng-star-inserted"))
        n_seats_available += len([1 for x in row.find_elements(By.XPATH, './/span') if x.get_attribute("class") == 'seat current'])
    
    show_times[n]["TotalSeats"] = n_seats
    show_times[n]["SeatsAvailable"] = n_seats_available    
    show_times[n]["Cost"] = '/'.join(map(str, seat_prices))

    #close this seat and go back to shows listing
    btn = driver.find_element(By.CLASS_NAME, "ion-arrow-left-c")
    btn.click()

    return show_times

In [22]:
from functions import * 
driver = webdriver.Chrome()
open_booking_page("JAWAN", driver)
next_day_click(driver)

scroll_page(driver)
scroll_page(driver)
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
show_times=dict()
n = len(cinema_list[:])

for i in range(n):
    time.sleep(3)
    scroll_page(driver)
    cinema = get_cinema(driver, i)
    print(i, n, cinema.find_element(By.CLASS_NAME, "title").text)
    if cinema in [show_times[ix]["Theatre"] for ix in show_times.keys()]:
        continue
    cinema_click(cinema)
    index = len(show_times)
    slot = showtime_click(cinema, show_times, index)
    wait_banners(driver)
    seat_info(driver, show_times, index) 


0 24 INOX Central, Mantri Junction, JP Nagar
no bag
no skip
1 24 INOX Shree Garuda Swagath Mall, Jayanagar
no bag
no skip
2 24 PVR VEGA Bengaluru
no bag
3 24 PVR Nexus (Formerly Forum), Koramangala
no bag
4 24 PVR Superplex Forum Mall Kanakpura Road
no bag
no skip
5 24 INOX Garuda Mall, Magrath Road
no bag
no skip
6 24 Director'S Cut Rex Walk Bengaluru
no bag
no skip
7 24 INOX Lido Mall, Ulsoor
no bag
no skip
8 24 PVR Global Mall, Mysore Road, Bengaluru
no bag
no skip
9 24 PVR GT World Mall Magadi Road
10 24 PVR Soul Spirit Bengaluru
no bag
11 24 INOX Mantri Square, Malleshwaram
no bag
no skip
12 24 PVR Orion Bengaluru
no bag
13 24 PVR Vaishnavi Sapphire Bengaluru
no bag
14 24 PVR VR Bengaluru
no bag
no skip


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=116.0.5845.188)
Stacktrace:
	GetHandleVerifier [0x00007FF67F2E52A2+57122]
	(No symbol) [0x00007FF67F25EA92]
	(No symbol) [0x00007FF67F12E25D]
	(No symbol) [0x00007FF67F168E97]
	(No symbol) [0x00007FF67F15F25F]
	(No symbol) [0x00007FF67F186FDA]
	(No symbol) [0x00007FF67F15EB76]
	(No symbol) [0x00007FF67F1871F0]
	(No symbol) [0x00007FF67F19EB82]
	(No symbol) [0x00007FF67F186DB3]
	(No symbol) [0x00007FF67F15D2B1]
	(No symbol) [0x00007FF67F15E494]
	GetHandleVerifier [0x00007FF67F58EF82+2849794]
	GetHandleVerifier [0x00007FF67F5E1D24+3189156]
	GetHandleVerifier [0x00007FF67F5DACAF+3160367]
	GetHandleVerifier [0x00007FF67F376D06+653702]
	(No symbol) [0x00007FF67F26A208]
	(No symbol) [0x00007FF67F2662C4]
	(No symbol) [0x00007FF67F2663F6]
	(No symbol) [0x00007FF67F2567A3]
	BaseThreadInitThunk [0x00007FFE17777344+20]
	RtlUserThreadStart [0x00007FFE180426B1+33]


In [11]:
from functions import * 

driver = webdriver.Chrome()

open_booking_page("JAWAN", driver)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
time.sleep(3)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
driver.execute_script("window.scrollTo(0, 0);")
len(cinema_list)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
driver.execute_script("window.scrollTo(0, 0);")

show_times=dict()
n = len(cinema_list[:])
# n = 3
try:
    for i in range(n):
        time.sleep(2)
        #get cinema list
        cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
        try:
            cinema = cinema_list[i]
        except:
            break 

        print(i, n, cinema.find_element(By.CLASS_NAME, "title").text)

        #click on the cinema to get seating
        cinema_button = cinema.find_element(By.CLASS_NAME, "cinema-title")
        cinema_button.click()

        #get all the h4 elements containing Hindi-2D
        hindi_2d_elements = cinema.find_elements(By.XPATH, '//h4[contains(@class, "type-title") and contains(text(), "HINDI- 2D")]')
        for hindi_2d_element in hindi_2d_elements:
            if len(hindi_2d_element.text)==0:
                continue #there might be empty classes
            ul_element = hindi_2d_element.find_element(By.XPATH, './following-sibling::ul[@class="type-time-slots ng-star-inserted"]')
            
            #get time slots for Hindi 2D in that cinema
            slot_elements = ul_element.find_elements(By.XPATH, './/li[@class="ng-star-inserted"]') 
            for slot in slot_elements:
                n=len(show_times)
                #pick the first available slot
                if (len(slot.text.strip()) != 0) and (slot.find_element(By.XPATH, './/span').get_attribute("class").split("-")[1] != "default"):
                    show_times[n] = {
                        "Theatre" : cinema.find_element(By.CLASS_NAME, "title").text,
                        "Time" : slot.text.strip(),
                        "type" : slot.find_element(By.XPATH, './/span').get_attribute("class").split("-")[1]
                    } 
                    break
            break

        try:
            #open the seating
            slot.click()
            time.sleep(2)
            wait = WebDriverWait(driver, 1)
            loaded = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "skip-btn")))

            #press the skip button in case some promo comes
            skip_btn = driver.find_element(By.CLASS_NAME, "skip-btn")
            skip_btn.click()

        except:
            print("no offers")

        #wait for the T&C banner to load and click the close button 
        time.sleep(5)
        
        # wait = WebDriverWait(driver, 1)
        # loaded = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "modal-content-header")))

        close_class = driver.find_element(By.CLASS_NAME, "modal-content-header")
        btn = close_class.find_element(By.CLASS_NAME, "ion-android-close")
        btn.click()

        #extract seats info
        seat_prices_array = [title.find_element(By.XPATH, ".//span").text for title in driver.find_elements(By.CLASS_NAME, "seats-col") if title.find_element(By.XPATH, ".//span").get_attribute("class")=="area hshshs"]
        seat_prices = [int(float(re.search(r'(\d+\.\d+)', seat).group(1))) for seat in seat_prices_array]

        n_seats = 0
        n_seats_available = 0
        rows = driver.find_elements(By.CSS_SELECTOR, ".seats-row.ng-star-inserted")
        for row in rows:
            n_seats+= len(row.find_elements(By.CSS_SELECTOR, ".seats-col.ng-star-inserted"))
            n_seats_available += len([1 for x in row.find_elements(By.XPATH, './/span') if x.get_attribute("class") == 'seat current'])
        
        show_times[n]["TotalSeats"] = n_seats
        show_times[n]["SeatsAvailable"] = n_seats_available    
        show_times[n]["Cost"] = '/'.join(map(str, seat_prices))

        #close this seat and go back to shows listing
        btn = driver.find_element(By.CLASS_NAME, "ion-arrow-left-c")
        btn.click()
        wait = WebDriverWait(driver, 5)
        element_on_next_page = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")))
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
        driver.execute_script("window.scrollTo(0, 0);")

except:
    print("error")



error


In [1]:
from functions import * 
movie_name = "JAWAN"
theatres = create_cinemas_list("Bengaluru")
driver = webdriver.Chrome()
open_booking_page("JAWAN", driver)
shows = create_show_list(driver)
final_df = final_data(shows, theatres, movie_name, "Bengaluru")


0 23 INOX Garuda Yelahanka
no offers
1 0 PVR VEGA Bengaluru
2 1 PVR VEGA Bengaluru
3 2 PVR Nexus (Formerly Forum), Koramangala
4 3 PVR Vaishnavi Sapphire Bengaluru
5 4 INOX Garuda Mall, Magrath Road
no offers
error


In [28]:
import pandas as pd
import sqlite3
import os
basedir = os.getcwd()
parent_dict = os.path.dirname(basedir)
db_path = os.path.join(parent_dict, 'todo.db')
conn = sqlite3.connect(db_path)
final_df.fillna(0, inplace=True)
final_df.to_sql('movie_shows', conn, if_exists='replace', index=True)
conn.commit()
conn.close()


In [6]:
from functions import * 
driver = webdriver.Chrome()
open_booking_page("JAWAN", driver)
next_day_click(driver)

scroll_page(driver)
scroll_page(driver)
cinema_list = driver.find_elements(By.CSS_SELECTOR, ".cinema-holder.ng-star-inserted")
show_times=dict()
n = len(cinema_list[:])
print(n)

24


In [16]:
baggage_btn = driver.find_element(By.CLASS_NAME, "custom-popup-container")
baggage_btn.find_element(By.CSS_SELECTOR, ".custom-popup-button.ng-star-inserted").click()

In [27]:
shows = pd.DataFrame(show_times).T
final_df = final_data(shows, theatres, movie_name, "Bengaluru")